In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.signal import butter, filtfilt
from IPython.display import Audio

# --- Parâmetros gerais ---
fs       = 44100                 # taxa de amostragem
B        = 1500                  # meia-largura da banda (Hz)
ordem_lp = 6                     # ordem do passa-baixas (baseband)

# Centros das faixas (9–12, 12–15, 15–18 kHz)
fc_list  = [10.5e3, 13.5e3, 16.5e3]

# Arquivos de áudio originais (curtos, mono)
files    = ["whatoncewas.wav",
            "heavenknowsimmisearablenow.wav",
            "wonderwall.wav"]


In [ ]:
def lowpass(x, fs, fc, ordem):
    """
    Butterworth passa-baixas em fc Hz
    """
    wn = fc/(fs/2)
    b, a = butter(ordem, wn, btype='low')
    return filtfilt(b, a, x)

# Carregar e filtrar cada áudio para 0–B Hz
sinais_bb = []
for fname in files:
    m, fs_file = sf.read(fname)
    assert fs_file == fs
    # LPF para manter só 0–B Hz
    m_filt = lowpass(m, fs, B, ordem_lp)
    sinais_bb.append(m_filt)

# Vetor de tempo (mesmo comprimento para todos)
N = len(sinais_bb[0])
t = np.arange(N) / fs


In [ ]:
sinais_mod = []

for (m_bb, fc) in zip(sinais_bb, fc_list):
    # 1) Gera portadora com COS para manter fase coerente no receptor
    port = np.cos(2 * np.pi * fc * t)

    # 2) Modula (multiplicação ponto a ponto)
    s_mod = m_bb * port
    sinais_mod.append(s_mod)

    # 3) Checa FFT e áudio modulado
    freqs = np.fft.fftfreq(N, 1/fs)[:N//2]
    mag   = np.abs(np.fft.fft(s_mod))[:N//2]
    plt.figure(figsize=(6,3))
    plt.plot(freqs, mag)
    plt.title(f"FFT Modulado @ {fc/1000:.1f} kHz")
    plt.xlim(fc - 2*B, fc + 2*B)
    plt.xlabel("Hz"); plt.ylabel("Magn"); plt.grid(True)
    plt.show()
    display(Audio(s_mod, rate=fs))

# 4) Soma os três modulados (sinal no “ar”)
sinal_total = sum(sinais_mod)

# 5) FFT do sinal composto e playback
freqs = np.fft.fftfreq(N, 1/fs)[:N//2]
mag   = np.abs(np.fft.fft(sinal_total))[:N//2]
plt.figure(figsize=(8,3))
plt.plot(freqs, mag)
plt.title("FFT do Sinal Composto")
plt.xlim(8e3, 20e3)
plt.xlabel("Hz"); plt.ylabel("Magn"); plt.grid(True)
plt.show()
display(Audio(sinal_total, rate=fs))
